# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'RA/STSOL) D(ASGCAAELG  EKMEQBLNNT',
 'calculationPrice': 'tops',
 'open': 0,
 'openTime': None,
 'openSource': 'iiaolffc',
 'close': 0,
 'closeTime': None,
 'closeSource': 'ffocliia',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 130.69,
 'latestSource': 'IEX real time price',
 'latestTime': '11:33:35 AM',
 'latestUpdate': 1625332014363,
 'latestVolume': None,
 'iexRealtimePrice': 133.71,
 'iexRealtimeSize': 420,
 'iexLastUpdated': 1637850141085,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 136.3,
 'previousVolume': 97268859,
 'change': -0.68,
 'changePercent': -0.0052,
 'volume': None,
 'iexMarketPercent': 0.019496846836133185,
 'iexVolume': 544985,
 'avgTo

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ], index = my_columns),
            ignore_index = True
        )
final_dataframe        

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,145.121,49.47,N/A
1,AAL,22.758,-1.57,N/A
2,AAP,198.940,21.39,N/A
3,AAPL,132.630,29.29,N/A
4,ABBV,118.100,41.04,N/A
...,...,...,...,...
500,YUM,119.320,33.07,N/A
501,ZBH,159.859,58.83,N/A
502,ZBRA,521.510,44.5,N/A
503,ZION,55.430,6.56,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0][:50]
final_dataframe.reset_index(inplace = True, drop = True)

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:500000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,627.280,4.48,15
1,CINF,123.096,6.52,81
2,ZION,55.430,6.56,180
3,AFL,56.274,7.3,177
4,COO,394.060,8.21,25
5,UNM,31.100,8.22,321
6,HOLX,64.800,8.8,154
7,PGR,96.700,8.84,103
8,KIM,22.121,9.09,452
9,CE,161.500,9.16,61


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# Price-to-earnings ratio

pe_ratio = data[symbol]['quote']['peRatio'] 

# Price-to-book ratio

pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio

ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda_value = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda_value

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gp = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda_value = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try: 
            ev_to_gp = enterprise_value/gross_profit
        except TypeError:
            ev_to_gp = np.NaN
        
        try: 
            ev_to_ebitda = enterprise_value/ebitda_value
        except TypeError:
            ev_to_ebitda = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
            ], index = rv_columns),
            ignore_index = True
        )

In [13]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.325,N/A,48.6,N/A,9.21,N/A,7.6,N/A,30.460971,N/A,14.553619,N/A,N/A
1,AAL,23.470,N/A,-1.55,N/A,-1.87,N/A,1.2,N/A,-3.769999,N/A,3.185962,N/A,N/A
2,AAP,204.590,N/A,21.61,N/A,3.88,N/A,1.23,N/A,11.177757,N/A,2.692298,N/A,N/A
3,AAPL,131.480,N/A,30.64,N/A,32.21,N/A,6.97,N/A,22.020633,N/A,17.355597,N/A,N/A
4,ABBV,119.470,N/A,40.23,N/A,15.07,N/A,4.2,N/A,12.804019,N/A,8.422175,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.300,N/A,33.04,N/A,-4.54,N/A,6.13,N/A,23.462743,N/A,10.357973,N/A,N/A
501,ZBH,165.059,N/A,59.06,N/A,2.71,N/A,4.75,N/A,27.572541,N/A,8.100434,N/A,N/A
502,ZBRA,509.880,N/A,44.51,N/A,11.85,N/A,5.96,N/A,31.474775,N/A,12.884245,N/A,N/A
503,ZION,54.160,N/A,6.77,N/A,1.2,N/A,2.6,N/A,5.460239,N/A,2.599369,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,254.780,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,290.570,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.07,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,29.966,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.390,N/A,14.56,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,64.538,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,57.880,N/A,33.52,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,37.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,193.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2623.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for columns in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[columns].fillna(rv_dataframe[columns].mean(), inplace = True)

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio':'PB Percentile',
    'Price-to-Sales Ratio':'PS Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/GP':'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.325,N/A,48.60,0.805941,9.21,0.790099,7.60,0.79802,30.460971,0.815842,14.553619,0.811881,N/A
1,AAL,23.470,N/A,-1.55,0.120792,-1.87,0.085149,1.20,0.118812,-3.769999,0.037624,3.185962,0.138614,N/A
2,AAP,204.590,N/A,21.61,0.388119,3.88,0.516832,1.23,0.126733,11.177757,0.249505,2.692298,0.108911,N/A
3,AAPL,131.480,N/A,30.64,0.556436,32.21,0.958416,6.97,0.764356,22.020633,0.655446,17.355597,0.871287,N/A
4,ABBV,119.470,N/A,40.23,0.742574,15.07,0.879208,4.20,0.556436,12.804019,0.352475,8.422175,0.489109,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.300,N/A,33.04,0.59703,-4.54,0.083168,6.13,0.729703,23.462743,0.693069,10.357973,0.643564,N/A
501,ZBH,165.059,N/A,59.06,0.861386,2.71,0.374257,4.75,0.6,27.572541,0.786139,8.100434,0.473267,N/A
502,ZBRA,509.880,N/A,44.51,0.790099,11.85,0.833663,5.96,0.716832,31.474775,0.827723,12.884245,0.758416,N/A
503,ZION,54.160,N/A,6.77,0.130693,1.20,0.126733,2.60,0.362376,5.460239,0.057426,2.599369,0.10297,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,146.325,N/A,48.60,0.805941,9.21,0.790099,7.60,0.79802,30.460971,0.815842,14.553619,0.811881,0.804356
1,AAL,23.470,N/A,-1.55,0.120792,-1.87,0.085149,1.20,0.118812,-3.769999,0.037624,3.185962,0.138614,0.100198
2,AAP,204.590,N/A,21.61,0.388119,3.88,0.516832,1.23,0.126733,11.177757,0.249505,2.692298,0.108911,0.27802
3,AAPL,131.480,N/A,30.64,0.556436,32.21,0.958416,6.97,0.764356,22.020633,0.655446,17.355597,0.871287,0.761188
4,ABBV,119.470,N/A,40.23,0.742574,15.07,0.879208,4.20,0.556436,12.804019,0.352475,8.422175,0.489109,0.60396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,119.300,N/A,33.04,0.59703,-4.54,0.083168,6.13,0.729703,23.462743,0.693069,10.357973,0.643564,0.549307
501,ZBH,165.059,N/A,59.06,0.861386,2.71,0.374257,4.75,0.6,27.572541,0.786139,8.100434,0.473267,0.61901
502,ZBRA,509.880,N/A,44.51,0.790099,11.85,0.833663,5.96,0.716832,31.474775,0.827723,12.884245,0.758416,0.785347
503,ZION,54.160,N/A,6.77,0.130693,1.20,0.126733,2.60,0.362376,5.460239,0.057426,2.599369,0.10297,0.15604


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace = True, drop = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,31.120,N/A,8.06,0.134653,0.5950,0.089109,0.4828,0.023762,4.027973,0.047525,0.461141,0.009901,0.06099
1,MCK,197.098,N/A,-7.01,0.10297,-1504.0200,0.00396,0.1302,0.009901,7.284481,0.083168,2.625593,0.10495,0.06099
2,L,56.940,N/A,-421.00,0.00396,0.8773,0.09901,1.0800,0.09802,8.097853,0.120792,1.009772,0.027723,0.069901
3,AIV,7.900,N/A,-84.24,0.021782,2.2700,0.318812,-35.3200,0.00198,-46.937982,0.007921,-58.285433,0.005941,0.071287
4,AIG,51.460,N/A,-11.71,0.089109,0.7262,0.093069,1.0000,0.087129,6.314152,0.063366,0.980398,0.025743,0.071683
5,HFC,35.630,N/A,-37.18,0.049505,1.0500,0.110891,0.4914,0.025743,8.528520,0.140594,2.930906,0.126733,0.090693
6,ABC,122.890,N/A,-6.57,0.10495,-90.4900,0.007921,0.1290,0.007921,8.778245,0.146535,4.421466,0.217822,0.09703
7,HPQ,29.800,N/A,10.60,0.174257,-10.7800,0.031683,0.6070,0.037624,7.871290,0.112871,3.198132,0.140594,0.099406
8,AAL,23.470,N/A,-1.55,0.120792,-1.8700,0.085149,1.2000,0.118812,-3.769999,0.037624,3.185962,0.138614,0.100198
9,BA,246.400,N/A,-12.40,0.083168,-8.2700,0.039604,2.6500,0.372277,-19.089605,0.019802,-42.415606,0.007921,0.104554


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:500000


In [21]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

C:\Users\hp\Desktop\Projects\algorithmic-trading-python\env\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,31.120,321,8.06,0.134653,0.5950,0.089109,0.4828,0.023762,4.027973,0.047525,0.461141,0.009901,0.06099
1,MCK,197.098,50,-7.01,0.10297,-1504.0200,0.00396,0.1302,0.009901,7.284481,0.083168,2.625593,0.10495,0.06099
2,L,56.940,175,-421.00,0.00396,0.8773,0.09901,1.0800,0.09802,8.097853,0.120792,1.009772,0.027723,0.069901
3,AIV,7.900,1265,-84.24,0.021782,2.2700,0.318812,-35.3200,0.00198,-46.937982,0.007921,-58.285433,0.005941,0.071287
4,AIG,51.460,194,-11.71,0.089109,0.7262,0.093069,1.0000,0.087129,6.314152,0.063366,0.980398,0.025743,0.071683
5,HFC,35.630,280,-37.18,0.049505,1.0500,0.110891,0.4914,0.025743,8.528520,0.140594,2.930906,0.126733,0.090693
6,ABC,122.890,81,-6.57,0.10495,-90.4900,0.007921,0.1290,0.007921,8.778245,0.146535,4.421466,0.217822,0.09703
7,HPQ,29.800,335,10.60,0.174257,-10.7800,0.031683,0.6070,0.037624,7.871290,0.112871,3.198132,0.140594,0.099406
8,AAL,23.470,426,-1.55,0.120792,-1.8700,0.085149,1.2000,0.118812,-3.769999,0.037624,3.185962,0.138614,0.100198
9,BA,246.400,40,-12.40,0.083168,-8.2700,0.039604,2.6500,0.372277,-19.089605,0.019802,-42.415606,0.007921,0.104554


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['Price-to-Earnings Ratio', float_template],
    'E':['PE Percentile', percent_template],
    'F':['Price-to-Book Ratio', float_template],
    'G':['PB Percentile', percent_template],
    'H':['Price-to-Sales Ratio', float_template],
    'I':['PS Percentile', percent_template],
    'J':['EV/EBITDA', float_template],
    'K':['EV/EBITDA Percentile', percent_template],
    'L':['EV/GP', float_template],
    'M':['EV/GP Percentile', percent_template],
    'N':['RV Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [28]:
writer.save()